<a href="https://colab.research.google.com/github/BootCamp-BMA/colabs/blob/main/llm_llama_prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
import transformers
import torch
from sklearn.metrics import (
    classification_report,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix
)

In [ ]:
# Load preprocessed FASSILA data
train_df = pd.read_csv("/content/drive/MyDrive/Project Fin d'Etude - Master/data/preprocessed/fassila_train.csv")
val_df   = pd.read_csv("/content/drive/MyDrive/Project Fin d'Etude - Master/data/preprocessed/fassila_val.csv")
test_df  = pd.read_csv("/content/drive/MyDrive/Project Fin d'Etude - Master/data/preprocessed/fassila_test.csv")

# Load precomputed embeddings and labels
X_train = np.load("/content/drive/MyDrive/Project Fin d'Etude - Master/data/embeddings/X_train.npy")
X_test  = np.load("/content/drive/MyDrive/Project Fin d'Etude - Master/data/embeddings/X_test.npy")
y_train = train_df["label"].tolist()
y_test  = test_df["label"].tolist()


In [ ]:
# Load TinyLLaMA-1.1B Chat model
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tinyllama_pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def get_top_k_examples(query_embedding, k=3):
    sims = cosine_similarity([query_embedding], X_train)[0]
    top_indices = sims.argsort()[-k:][::-1]
    examples = []
    for idx in top_indices:
        text = train_df.iloc[idx]["text"]
        label = "Real" if y_train[idx] == 0 else "Fake"
        examples.append((text, label))
    return examples


In [ ]:
def build_prompt(examples, test_text):
    prompt = ""
    for i, (text, label) in enumerate(examples):
        prompt += f"Example {i+1}:\nText: \"{text}\"\nLabel: {label}\n\n"
    prompt += "---\nNow classify the following post:\n"
    prompt += f"Text: \"{test_text}\"\n"
    prompt += "Is this post Real or Fake?"
    return prompt


In [ ]:
def classify_with_tinyllama(prompt):
    full_prompt = f"<|system|>\nYou are a helpful assistant that classifies Arabic Algerian social media posts as 'Real' or 'Fake'.\n<|user|>\n{prompt}\n<|assistant|>"

    output = tinyllama_pipeline(full_prompt, max_new_tokens=128)[0]["generated_text"]
    output_lower = output.lower()

    if "real" in output_lower:
        return 0
    elif "fake" in output_lower:
        return 1
    else:
        return -1  # Unknown / failed to classify


In [ ]:
y_pred = []

for i in tqdm(range(len(test_df)), desc="Running TinyLLaMA"):
    test_text = test_df.iloc[i]["text"]
    test_embedding = X_test[i]

    examples = get_top_k_examples(test_embedding, k=3)
    prompt = build_prompt(examples, test_text)
    prediction = classify_with_tinyllama(prompt)

    y_pred.append(prediction)


Running TinyLLaMA: 100%|██████████| 1010/1010 [35:45<00:00,  2.12s/it]


In [ ]:
print(classification_report(y_test, y_pred, target_names=["Real", "Fake"]))


              precision    recall  f1-score   support

        Real       0.54      1.00      0.70       541
        Fake       0.00      0.00      0.00       469

    accuracy                           0.54      1010
   macro avg       0.27      0.50      0.35      1010
weighted avg       0.29      0.54      0.37      1010



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
